In [ ]:
%%python -V

Python 3.11.13


In [ ]:
%pip install "git+https://github.com/coqui-ai/TTS.git@dev"
%pip install torch torchaudio
%pip install datasets
%pip install peft
%pip install accelerate
%pip install "transformers==4.46.3"

  Cloning https://github.com/coqui-ai/TTS.git (to revision dev) to /tmp/pip-req-build-43o7y34k
  Running command git clone --filter=blob:none --quiet https://github.com/coqui-ai/TTS.git /tmp/pip-req-build-43o7y34k
  Resolved https://github.com/coqui-ai/TTS.git to commit dbf1a08a0d4e47fdad6172e433eeb34bc6b13b4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 123.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.2
    Uninstalling transformers-4.53.2:
      Successfull

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import numpy as np
from pathlib import Path
import json
from typing import Dict, List, Tuple
import pandas as pd
from TTS.api import TTS
from TTS.tts.configs.xtts_config import XttsConfig, XttsAudioConfig, XttsArgs
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.models.xtts import Xtts
from TTS.utils.audio import AudioProcessor
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
from IPython.display import Audio, display
import warnings
warnings.filterwarnings('ignore')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


## Serialization
Pytorch added a `weights_only` parameter, but the tts library doesn't support it. Therefore, we have to do some patchwork before training or inference.

In [ ]:
import torch.serialization

torch.serialization.add_safe_globals([XttsConfig, XttsAudioConfig, BaseDatasetConfig, XttsArgs])

device = "cuda" if torch.cuda.is_available() else "cpu"

tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# # Example generation
# tts.tts_to_file(
#     text="I totally agree with what you're saying.",
#     speaker_wav="reference.wav",
#     language="en",
#     file_path="output.wav"
# )

# print("Done! Check output_chinese.wav")

 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.87G/1.87G [00:18<00:00, 102MiB/s]
100%|██████████| 1.87G/1.87G [00:19<00:00, 95.9MiB/s]
4.37kiB [00:00, 8.29kiB/s]

361kiB [00:00, 664kiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 56.2iB/s]
 49%|████▉     | 3.84M/7.75M [00:00<00:00, 38.4MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


GPT2InferenceModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


## Regular Fine Tuning
Code is adapted from the coqui-ai tts library recipe for xTTS-v2.

In [ ]:
from trainer import Trainer, TrainerArgs

from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig
from TTS.utils.manage import ModelManager

RUN_NAME = "GPT_XTTS_v2.0_CHINESE_FINE_TUNING"
PROJECT_NAME = "XTTS_Fine_Tuning_Trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

OUT_PATH = os.path.join(os.getcwd(), "run", "training")

OPTIMIZER_WD_ONLY_ON_WEIGHTS = True
START_WITH_EVAL = True
BATCH_SIZE = 3
GRAD_ACUMM_STEPS = 84

config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="tw_zh_dataset",
    path="/content/drive/MyDrive/493/ljs-mini",
    meta_file_train="/content/drive/MyDrive/493/ljs-mini/metadata.csv",
    language="zh-cn",
)

DATASETS_CONFIG_LIST = [config_dataset]

CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)


DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)


TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))

if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )


SPEAKER_REFERENCE = [
    "/content/drive/MyDrive/493/taiwanese_reference.wav"  # speaker reference to be used in training test sentences
]
LANGUAGE = config_dataset.language


def main():
    model_args = GPTArgs(
        max_conditioning_length=132300,
        min_conditioning_length=66150,
        debug_loading_failures=False,
        max_wav_length=255995,
        max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True
    )

    audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000)

    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="GPT xTTS training",
        epochs=30,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        print_eval=False,
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,
        lr_scheduler="MultiStepLR",
        lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
        test_sentences=[
            {
                "text": "你好，这是一个中文语音合成的测试。",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": "zh-cn"
            },
            {
                "text": "我坐计程车去用电脑软体。",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": "zh-cn"
            },
            {
                "text": "我知道是谁吃的，是不是张先生？",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": "zh-cn"
            }
        ],
    )

    model = GPTTrainer.init_from_config(config)

    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    trainer = Trainer(
        TrainerArgs(
            restore_path=None,
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()

if __name__ == "__main__":
  main()

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 8
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_10+54AM-0000000


>> DVAE weights restored from: /content/run/training/XTTS_v2.0_original_model_files/dvae.pth
 | > Found 544 files in /content/drive/MyDrive/493/ljs-mini



 > Model has 518442047 parameters

 > EPOCH: 0/30
 --> /content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_10+54AM-0000000

 > EVALUATION 



 > Filtering invalid eval samples!!
 > Total eval samples after filtering: 5


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.06465387344360352 (-0.0021486282348632812)
     | > avg_loss_text_ce: 0.03010459430515766 (-3.176368772983551e-05)
     | > avg_loss_mel_ce: 2.2270195484161377 (-0.0072345733642578125)
     | > avg_loss: 2.257124185562134 (-0.00726628303527832)

 > BEST MODEL : /content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_10+54AM-0000000/best_model_4860.pth

 > EPOCH: 28/30
 --> /content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_10+54AM-0000000

 > TRAINING (2025-12-09 11:49:38) 

   --> TIME: 2025-12-09 11:49:59 -- STEP: 40/180 -- GLOBAL_STEP: 4900
     | > loss_text_ce: 0.0348258838057518  (0.03256304478272796)
     | > loss_mel_ce: 2.9332213401794434  (2.7588285952806473)
     | > loss: 0.035333894193172455  (0.03323085359297693)
     | > current_lr: 5e-06 
     | > step_time: 0.3063  (0.29358007311821)
     | > loader_time: 0.0083  (0.008280444145202638)


   --> TIME: 2025-12-09 11:50:25 -- STEP: 90/180 -- GLOB

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.06978058815002441 (+0.0051267147064208984)
     | > avg_loss_text_ce: 0.03005998581647873 (-4.460848867893219e-05)
     | > avg_loss_mel_ce: 2.2202320098876953 (-0.006787538528442383)
     | > avg_loss: 2.2502920627593994 (-0.006832122802734375)

 > BEST MODEL : /content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_10+54AM-0000000/best_model_5040.pth

 > EPOCH: 29/30
 --> /content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_10+54AM-0000000

 > TRAINING (2025-12-09 11:51:44) 

   --> TIME: 2025-12-09 11:51:50 -- STEP: 10/180 -- GLOBAL_STEP: 5050
     | > loss_text_ce: 0.029666868969798088  (0.03320489432662725)
     | > loss_mel_ce: 3.241757392883301  (2.849664831161499)
     | > loss: 0.038945529609918594  (0.03431987855583429)
     | > current_lr: 5e-06 
     | > step_time: 0.3187  (0.2820259094238281)
     | > loader_time: 0.0072  (0.008988046646118164)


   --> TIME: 2025-12-09 11:52:16 -- STEP: 60/180 -- G

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.11164093017578125 (+0.041860342025756836)
     | > avg_loss_text_ce: 0.030012916773557663 (-4.7069042921066284e-05)
     | > avg_loss_mel_ce: 2.213446855545044 (-0.006785154342651367)
     | > avg_loss: 2.243459701538086 (-0.0068323612213134766)

 > BEST MODEL : /content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_10+54AM-0000000/best_model_5220.pth


## Regular Fine Tuning Inference
Sanity check to make sure that our model actually works

In [ ]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

CHECKPOINT_DIR = "/content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_10+54AM-0000000"

SPECIFIC_CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "best_model.pth")

SPEAKER_REFERENCE = "/content/drive/MyDrive/493/taiwanese_reference.wav"
OUTPUT_WAV_PATH = "output_best.wav"

config = XttsConfig()

config.load_json(os.path.join(CHECKPOINT_DIR, "config.json"))

model = Xtts.init_from_config(config)

model.load_checkpoint(
    config,
    checkpoint_dir=CHECKPOINT_DIR,
    checkpoint_path=SPECIFIC_CHECKPOINT_PATH,
    vocab_path=os.path.join(CHECKPOINTS_OUT_PATH, "vocab.json"),
    eval=True,
    use_deepspeed=False
)

model.cuda()

Loading model...
Generating audio...
Saved to output_best.wav


In [ ]:
outputs = model.synthesize(
    "没想到今天的垃圾车这么早就来了，而且还刚好是礼拜三。",
    config,
    speaker_wav=SPEAKER_REFERENCE,
    gpt_cond_len=3,
    language="zh-cn",
)

torchaudio.save(OUTPUT_WAV_PATH, torch.tensor(outputs["wav"]).unsqueeze(0), 24000)
print(f"Saved to {OUTPUT_WAV_PATH}")

Generating audio...
Saved to output_best.wav


## LoRA Fine Tuning
Again, training code is adapted from coqui-ai's tts library.

Note: LoRA adapters aren't natively supported by the tts library, so we have to do some wacky monkey patching to make it work.
1. Instead of using the model returned by the library's GPTTrainer, we have to wrap the underlying gpt layer with a LoRA adapter from peft.
2. We have to sidestep the test run (hence the safe_test_run function) which is fine since we're not using tensorboard anyways and don't need it.

In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager
from peft import LoraConfig, get_peft_model

RUN_NAME = "GPT_XTTS_v2.0_CHINESE_LoRA"
PROJECT_NAME = "XTTS_LoRA_Trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

OUT_PATH = os.path.join(os.getcwd(), "run", "training")

OPTIMIZER_WD_ONLY_ON_WEIGHTS = True
START_WITH_EVAL = True
BATCH_SIZE = 3
GRAD_ACUMM_STEPS = 84

config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="tw_zh_dataset",
    path="/content/drive/MyDrive/493/ljs-mini",
    meta_file_train="/content/drive/MyDrive/493/ljs-mini/metadata.csv",
    language="zh-cn",
)

DATASETS_CONFIG_LIST = [config_dataset]

CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)


DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)


TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))

if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )


SPEAKER_REFERENCE = [
    "/content/drive/MyDrive/493/taiwanese_reference.wav"
]
LANGUAGE = config_dataset.language

def main():
    model_args = GPTArgs(
        max_conditioning_length=132300,
        min_conditioning_length=66150,
        debug_loading_failures=False,
        max_wav_length=255995,
        max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True
    )

    audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000)

    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="GPT xTTS training (LoRA)",
        epochs=30,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        print_eval=False,
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=2e-04,
        lr_scheduler="MultiStepLR",
        lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
        test_sentences=[
            {
                "text": "你好，这是一个中文语音合成的测试。",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": "zh-cn"
            },
            {
                "text": "我坐计程车去用电脑软体。",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": "zh-cn"
            },
            {
                "text": "我知道是谁吃的，是不是张先生？",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": "zh-cn"
            }
        ],
    )

    def safe_test_run(*args):
      print("\tSkipping audio generation during training to prevent LoRA crash.")
      return {"audios": {}, "figures": {}}

    GPTTrainer.test_run = safe_test_run

    model = GPTTrainer.init_from_config(config)

    print("Injecting LoRA adapters")
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["c_attn", "c_proj"],
        lora_dropout=0.05,
        bias="none",
    )

    model.xtts.gpt = get_peft_model(model.xtts.gpt, lora_config)
    model.xtts.gpt.print_trainable_parameters()

    original_gpt = model.xtts.gpt.base_model.model
    for attr_name in dir(original_gpt):
        if not attr_name.startswith("__") and not hasattr(model.xtts.gpt, attr_name):
            attr_val = getattr(original_gpt, attr_name)
            if callable(attr_val):
                setattr(model.xtts.gpt, attr_name, attr_val)
                print(f" > Patched method: {attr_name}")

    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    trainer = Trainer(
        TrainerArgs(
            restore_path=None,
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()

if __name__ == "__main__":
  main()

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 8
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_06+12PM-0000000


>> DVAE weights restored from: /content/run/training/XTTS_v2.0_original_model_files/dvae.pth
Injecting LoRA adapters
trainable params: 2,703,360 || all params: 443,721,923 || trainable%: 0.6092
 | > Found 544 files in /content/drive/MyDrive/493/ljs-mini



 > Model has 76456604 parameters

 > EPOCH: 0/30
 --> /content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_06+12PM-0000000

 > EVALUATION 



 > Filtering invalid eval samples!!
 > Total eval samples after filtering: 5



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.07132267951965332 (+0)
     | > avg_loss_text_ce: 0.03366192430257797 (+0)
     | > avg_loss_mel_ce: 3.362276077270508 (+0)
     | > avg_loss: 3.395937919616699 (+0)


 > EPOCH: 1/30
 --> /content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_06+12PM-0000000

 > TRAINING (2025-12-09 18:12:46) 


...

 > Skipping audio generation during training to prevent LoRA crash.
 > Skipping audio generation during training to prevent LoRA crash.



 > EPOCH: 28/30
 --> /content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_06+12PM-0000000

 > TRAINING (2025-12-09 18:46:38) 

   --> TIME: 2025-12-09 18:46:53 -- STEP: 40/180 -- GLOBAL_STEP: 4900
     | > loss_text_ce: 0.03310401365160942  (0.032831060141325)
     | > loss_mel_ce: 3.0341036319732666  (2.8860660552978517)
     | > loss: 0.03651437535881996  (0.034748775977641345)
     | > current_lr: 0.0002 
     | > step_time: 0.2397  (0.23942525386810304)
     | > loader_time: 0.0099  (0.009962046146392824)


   --> TIME: 2025-12-09 18:47:10 -- STEP: 90/180 -- GLOBAL_STEP: 4950
     | > loss_text_ce: 0.03482534736394882  (0.03315388212601346)
     | > loss_mel_ce: 3.017744779586792  (2.9120740387174826)
     | > loss: 0.03634012117981911  (0.0350622379531463)
     | > current_lr: 0.0002 
     | > step_time: 0.255  (0.24444190661112467)
     | > loader_time: 0.0101  (0.009789525138007276)


   --> TIME: 2025-12-09 18:47:29 -- STEP: 140/180 -- GLOBAL_STEP: 5000
     | > l

 > Skipping audio generation during training to prevent LoRA crash.
 > Skipping audio generation during training to prevent LoRA crash.



   --> TIME: 2025-12-09 18:47:48 -- STEP: 10/180 -- GLOBAL_STEP: 5050
     | > loss_text_ce: 0.0302448607981205  (0.0331746194511652)
     | > loss_mel_ce: 3.337573528289795  (2.95085985660553)
     | > loss: 0.0400930754840374  (0.03552421983331442)
     | > current_lr: 0.0002 
     | > step_time: 0.2618  (0.23493587970733643)
     | > loader_time: 0.0097  (0.009970521926879883)


   --> TIME: 2025-12-09 18:48:05 -- STEP: 60/180 -- GLOBAL_STEP: 5100
     | > loss_text_ce: 0.027097152546048164  (0.03244521661351124)
     | > loss_mel_ce: 2.511887311935425  (2.962064292033513)
     | > loss: 0.030226007103919983  (0.03564892308786513)
     | > current_lr: 0.0002 
     | > step_time: 0.2369  (0.239552640914917)
     | > loader_time: 0.011  (0.009958338737487792)


   --> TIME: 2025-12-09 18:48:22 -- STEP: 110/180 -- GLOBAL_STEP: 5150
     | > loss_text_ce: 0.03912338614463806  (0.032920503497801026)
     | > loss_mel_ce: 2.7353758811950684  (2.943203201077201)
     | > loss: 0.033029753

 > Skipping audio generation during training to prevent LoRA crash.
 > Skipping audio generation during training to prevent LoRA crash.


## LoRA Inference
Similar to the regular fine tuning inference code, but again, since the tts library doesn't support LoRA adapters we have to inject peft.

In [ ]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from peft import LoraConfig, get_peft_model

RUN_DIR = "/content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_06+12PM-0000000"

FINE_TUNED_PTH = os.path.join(RUN_DIR, "best_model.pth")

BASE_MODEL_DIR = "/content/run/training/XTTS_v2.0_original_model_files/"

SPEAKER_REFERENCE = "/content/drive/MyDrive/493/taiwanese_reference.wav"
OUTPUT_WAV_PATH = "lora_result.wav"

config = XttsConfig()
config.load_json(os.path.join(RUN_DIR, "config.json"))

model = Xtts.init_from_config(config)

model.load_checkpoint(
    config,
    checkpoint_dir=BASE_MODEL_DIR,
    vocab_path=os.path.join(BASE_MODEL_DIR, "vocab.json"),
    eval=True,
    use_deepspeed=False
)
model.cuda()

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.05,
    bias="none",
)

model.gpt = get_peft_model(model.gpt, lora_config)

checkpoint = torch.load(FINE_TUNED_PTH, map_location="cuda")

if "model" in checkpoint:
    state_dict = checkpoint["model"]
else:
    state_dict = checkpoint

model.load_state_dict(state_dict, strict=False)

model.eval()

original_gpt = model.gpt.base_model.model
if not hasattr(model.gpt, "gpt_inference"):
    model.gpt.gpt_inference = original_gpt.gpt_inference

outputs = model.synthesize(
    "我知道是谁吃的，是不是张先生？",
    config,
    speaker_wav=SPEAKER_REFERENCE,
    gpt_cond_len=3,
    language="zh-cn",
)

torchaudio.save(OUTPUT_WAV_PATH, torch.tensor(outputs["wav"]).unsqueeze(0), 24000)
print(f"Saved to {OUTPUT_WAV_PATH}")

 > Loading base model...
 > Re-creating LoRA structure...
 > Loading fine-tuned weights from /content/run/training/GPT_XTTS_v2.0_CHINESE_LoRA-December-09-2025_06+12PM-0000000/best_model.pth...
 > Generating audio...
 > Saved to lora_result.wav
